In [7]:
import pandas as pd
import requests
import json

KOBERT_URL = 'http://3.35.8.82:5000/kobert?text='
BERT_URL = 'http://3.35.8.82:5000/bert?text='

In [51]:
movie = pd.read_csv('movie_datasets.csv', index_col=False)
movie2 = pd.read_csv('af_kmdb.csv')
book = pd.read_csv('book_datasets.csv')
book2 = pd.read_csv('naver_books.csv')
music = pd.read_csv('music_datasets.csv')
music2 = pd.read_csv('music_datasets_4000.csv')


In [52]:
movie.head()

,Unnamed: 0,title,category,genre,year,direct,actors,content,emotions
0,0,에브리씽 유브 갓 하우 두 유 노우?,극영화,"로맨스,코메디",2010,제임스 L. 브룩스,제임스 L. 브룩스,"소프트볼,국가대표,탈락,투수,중역,회계,부정,누명","{'공포': 0.0029, '놀람': 0.0111, '분노': 0.1089, '슬픔..."
1,1,S.W.A.T.: 파이어 파이트,극영화,액션,2011,베니 붐,리드 스타이너,"동료,킬러,LA,경찰특공대,심리학자,인질,복수","{'공포': 0.4642, '놀람': 0.0396, '분노': 0.2574, '슬픔..."
2,2,넥스트 어벤져 : 히어로 오브 투모로우,애니메이션,"어드벤처,액션",2012,"제이 올리바,개리 해틀","그레그 존슨,크랙 카일","어벤져,아이언맨,출생의비밀,헐크","{'공포': 0.0406, '놀람': 0.7997, '분노': 0.0041, '슬픔..."
3,3,스콧 필그림,극영화,"로맨스,코메디,판타지,액션",2010,에드가 라이트,"에드가 라이트,마이클 바콜","토론토,록밴드,이상형,데이트,결투,전남자친구","{'공포': 0.0012, '놀람': 0.0184, '분노': 0.001, '슬픔'..."
4,4,복수자,극영화,"드라마,범죄,스릴러,액션",2010,조지 틸만 주니어,"토니 게이튼,조 게이튼","은행털이,감옥,복수,은퇴,형사,킬러","{'공포': 0.0175, '놀람': 0.0131, '분노': 0.0334, '슬픔..."


In [10]:
def getEmotionsFromAPI(model='kobert', text=''):
    url = KOBERT_URL if model=='kobert' else BERT_URL
    res = requests.get(url + text).text
    return json.loads(res)

def getDistance(user_emo, media_emo):
    total = 0
    for key in user_emo.keys():
        n1 = user_emo[key]
        n2 = media_emo[key]
        total += (n1-n2)**2
    return total

In [11]:
from tqdm.notebook import tqdm
def getSeries(series, model='kobert'):
    tmp = []
    for row in tqdm(series):
        tmp.append(getEmotionsFromAPI(model=model, text=row))
    print('len:', len(tmp))
    return pd.Series(tmp)
# book['emotions'] = book['description'].apply(lambda x:getEmotionsFromAPI('kobert', x))
    

In [43]:
book['emotions'] = getSeries(book['description'], 'kobert')
# music['emotions'] = getSeries(music['new_lyrics'], 'bert')
# movie['emotions'] = getSeries(movie['content'], 'kobert')

  0%|          | 0/1021 [00:00<?, ?it/s]

len: 1021


In [49]:
book.drop(columns=['Unnamed: 0'])
movie.drop(columns=['Unnamed: 0'])
music.drop(columns=['Unnamed: 0'])

,artists,featured_artists,title,genre,lyrics,url,new_lyrics,emotions
0,Eminem,NaN,Rap God,rap,"Rap God Lyrics[Intro]\r\n""Look, I was gonna go...",https://genius.com/Eminem-rap-god-lyrics,"""Look, I was gonna go easy on you not to hurt ...","{'공포': 0.0702, '기쁨': 0.0297, '놀람': 0.0528, '분노..."
1,Cardi B,Megan Thee Stallion,WAP,rap,"WAP Lyrics[Intro: Cardi B, Al ""T"" McLaran & Me...",https://genius.com/Cardi-b-wap-lyrics,Whores in this house There's some whores in th...,"{'공포': 0.0016, '기쁨': 0.0476, '놀람': 0.0061, '분노..."
2,Kendrick Lamar,NaN,HUMBLE.,rap,HUMBLE. Lyrics[Intro]\r\nNobody pray for me\r\...,https://genius.com/Kendrick-lamar-humble-lyrics,Nobody pray for me It been that day for me Way...,"{'공포': 0.0029, '기쁨': 0.0347, '놀람': 0.0113, '분노..."
3,Migos,Lil Uzi Vert,Bad and Boujee,rap,Bad and Boujee Lyrics[Intro: Offset & Future]\...,https://genius.com/Migos-bad-and-boujee-lyrics,"You know, young rich niggas You know somethin'...","{'공포': 0.0009, '기쁨': 0.4118, '놀람': 0.0055, '분노..."
4,Travis Scott,Drake,SICKO MODE,rap,SICKO MODE Lyrics[Part I]\r\n\r\n[Intro: Drake...,https://genius.com/Travis-scott-sicko-mode-lyrics,"Astro, yeah Sun is down, freezin' cold That'...","{'공포': 0.002, '기쁨': 0.019, '놀람': 0.0092, '분노':..."
...,...,...,...,...,...,...,...,...
1301,Michael Jackson,NaN,Thriller,soul,Thriller Lyrics[Verse 1]\r\nIt's close to midn...,https://genius.com/Michael-jackson-thriller-ly...,It's close to midnight Something evil's lurkin...,"{'공포': 0.7768, '기쁨': 0.0115, '놀람': 0.3331, '분노..."
1302,Miguel,"J. Cole,Salaam Remi",Come Through and Chill,soul,Come Through and Chill Lyrics[Intro: Miguel & ...,https://genius.com/Miguel-come-through-and-chi...,"Uh, trust Pick up the phone, show me it was re...","{'공포': 0.0031, '기쁨': 0.2415, '놀람': 0.0043, '분노..."
1303,Bruno Mars,NaN,Grenade,soul,"Grenade Lyrics[Verse 1]\r\nEasy come, easy go\...",https://genius.com/Bruno-mars-grenade-lyrics,"Easy come, easy go That's just how you live, o...","{'공포': 0.0026, '기쁨': 0.0721, '놀람': 0.0302, '분노..."
1304,Michael Jackson,NaN,Man in the Mirror,soul,Man in the Mirror Lyrics[Verse 1]\r\nI'm gonna...,https://genius.com/Michael-jackson-man-in-the-...,I'm gonna make a change For once in my life Go...,"{'공포': 0.0015, '기쁨': 0.3345, '놀람': 0.4141, '분노..."


In [50]:
movie.to_csv('movie_datasets.csv', index=False)
book.to_csv('book_datasets.csv', index=False)
music.to_csv('music_datasets.csv', index=False)

In [ ]:
print(movie['title'][0])
row = movie['content'][0]
print(row)
print(getEmotionsFromAPI('kobert', row))

에브리씽 유브 갓 하우 두 유 노우?
소프트볼,국가대표,탈락,투수,중역,회계,부정,누명
{'공포': 0.0029, '놀람': 0.0111, '분노': 0.1089, '슬픔': 0.04, '중립': 0.0386, '행복': 0.003, '혐오': 0.7956}
